In [1]:
%pip install tqdm
%pip install scipy

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import json
import numpy as np
from scipy import stats
from datetime import datetime
import math
from time import sleep
from tqdm import tqdm
from scipy.spatial import cKDTree

In [3]:
#Pre-process combined focus

focus = pd.read_csv("combined_focos.csv")
focus.drop(labels=['frp'], inplace=True, axis=1)
focus.dropna(inplace=True)
focus.reset_index(drop=True, inplace=True)
focus.isna().sum()

latitude                 0
longitude                0
data_pas                 0
satelite                 0
pais                     0
estado                   0
municipio                0
bioma                    0
numero_dias_sem_chuva    0
precipitacao             0
risco_fogo               0
id_area_industrial       0
dtype: int64

In [9]:
stations = pd.read_csv("../INMET_weather/stations.csv")

In [10]:
# Preprocessar coordenadas das estações
stations['lat'] = stations['lat'].str.replace(',', '.').astype(float)
stations['lon'] = stations['lon'].str.replace(',', '.').astype(float)

# Construir KDTree para estações
station_coords = stations[['lat', 'lon']].values
tree = cKDTree(station_coords)

# Coordenadas dos focos
focus_coords = focus[['latitude', 'longitude']].values

# Encontrar índices das estações mais próximas
distances, indices = tree.query(focus_coords)

# Criar DataFrame final
filtered_focus = pd.DataFrame({
    'date': pd.to_datetime(focus['data_pas']),
    'city': stations['city_station'].values[indices],
    'biome': focus['bioma'],
    'days_without_rain': focus['numero_dias_sem_chuva'],
    'precipitation': focus['precipitacao'],
    'fire_risk': focus['risco_fogo']
})

print(filtered_focus)

                        date               city     biome  days_without_rain  \
0        2018-03-01 23:05:45          BOA VISTA  Amazônia                0.0   
1        2018-06-09 16:54:00       PORTO ALEGRE     Pampa                2.0   
2        2018-06-09 14:20:00      DTCEA VILHENA  Amazônia                7.0   
3        2018-06-09 14:20:00      DTCEA VILHENA  Amazônia                7.0   
4        2018-06-09 14:20:00  S.J. DO RIO CLARO   Cerrado               18.0   
...                      ...                ...       ...                ...   
49141511 2023-07-27 23:23:06            TUCURUI  Amazônia               25.0   
49141512 2023-07-27 23:23:11           TOME ACU  Amazônia               14.0   
49141513 2023-07-27 23:23:11           TOME ACU  Amazônia               13.0   
49141514 2023-07-27 23:23:22           TOME ACU  Amazônia                9.0   
49141515 2023-07-27 23:23:22           TOME ACU  Amazônia               10.0   

          precipitation  fire_risk  
0 

In [ ]:
focus_train = filtered_focus[filtered_focus["data_pas"].apply(lambda x: int(x[:4]))<=2020]
focus_val = filtered_focus[filtered_focus["data_pas"].apply(lambda x: int(x[:4]))>2020]
focus_val = focus_val[focus_val["data_pas"].apply(lambda x: int(x[:4]))<=2022]
focus_test = filtered_focus[filtered_focus["data_pas"].apply(lambda x: int(x[:4]))>2022]

In [ ]:
inmet_test = pd.read_csv("inmet_test.csv")
inmet_train = pd.read_csv("inmet_train.csv")
inmet_val = pd.read_csv("inmet_val.csv")